New collect and redistribyte dynamics

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import random as random
import csv
import heapq
from tqdm import tqdm
from math import factorial
import time

%matplotlib osx
font = {'family' : 'SimSun',
        'weight' : 'bold',
         'size'   : '8'}
plt.rc('font', **font)               # 步骤一（设置字体的更多属性）
plt.rc('axes', unicode_minus=False)  # 步骤二（解决坐标轴负数的负号显示问题）

In [2]:
# Hyper parameters
EPISODES = 1 # times every setting is replicated
# random.seed(1)
WriteFile = False
#WriteFile = True
FileAdd = '/Users/valarian/SJTU/SJTU/毕业论文/performanceA2M6HigherMu.csv'
WARMTIME = 0
RUNTIME = 200

In [3]:
Instance = 4
np.random.seed(Instance)
random.seed(0)

#A = 10
#M = 50
A, M = 2, 6
FileAdd = 'C:\\Rebalancing\\2020-8-31\\result\\A'+str(A)+'M'+str(M)

def getPij(a):
    temp = np.log1p(np.random.rand(A,A))
    return (temp/sum(temp)).T
Pij = getPij(A)
ArrLst = 6*np.random.rand(A)
Beta = 0.3

Tau = 1.0
C = 3
Mu = 1
N = 1

RhoMtx = [[1/(abs(j-i)+1) for i in range(A)] for j in range(A)]

In [4]:
# test the simulation length needed

class Model():
    '''
    This is the central model
    '''
    # initiate the parameters in this function
    def __init__(self, A, M, Pij, ArrLst, RhoMtx, Beta, B_, Gamma, Mu, N, Delta, D_):     
        self.timeLimit = WARMTIME + RUNTIME
        self.areas = list(range(A))
        #self.epi = 0
        
        #self.Performance = [0] * EPISODES
        self.A, self.M, self.Pij, self.ArrLst, self.RhoMtx, self.Beta, self.B_, self.Gamma, self.Mu, self.N, self.Delta, self.D_ = \
            A, M, Pij, ArrLst, RhoMtx, Beta, B_, Gamma, Mu, N, Delta, D_
        
        self.color = ['r']
        
        def get_target_number():
            A, M, D_, ArrLst = self.A, self.M, self.D_, self.ArrLst
            arr = np.array(ArrLst)/sum(ArrLst)
            num_dis = [int(M*x) for x in arr]
            left = M - sum(num_dis)
            for i in np.argsort(ArrLst)[:left]:
                num_dis[i] += 1
            return num_dis
        self.num_dis = get_target_number()
        self.arr_rank = np.argsort(ArrLst)[::-1]
        #print(self.num_dis, self.ArrLst, self.arr_rank)
    def reset(self):
        self.T = 0 # time cursor
        self.formerT = 0
        
        # serve performance parameters
        self.normalBikes, self.brokenBikes = M, 0
        # maintainance performance parameters
        self.idleRate, self.BP, self.RC, self.DP = 0, 0, 0, 0
        self.arrivals, self.lostCustomers = 0, 0
        # indicators of time stamp
        [self.nt, self.bt, self.it, self.bpt, self.rct, self.dpt] = [0]*6
        self.enormalBikes, self.ebrokenBikes = 0, 0
        # maintainance performance parameters
        self.eidleRate, self.eBP, self.eRC, self.eDP = 0, 0, 0, 0
        self.Result = [0.0] * 7
        
        self.state1 = [int(M/A)]*A + [0]*3 # Nis, BP, FC, and DP
        self.state2 = [[0]*A for i in range(A)] # Rijs
        self.F = [] # time to be empty for fixing queue
        heapq.heapify(self.F)
        
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(A):
            heapq.heappush(self.scheduler, [random.expovariate(ArrLst[i]), -1, i, i])
        self.stateRecord = self.state1[:A] + self.state2[0] + self.state2[1] + self.state1[-3:]
        
        # save state for drawing
        self.hn, self.hl, self.hi = [[],[]], [[],[]], [[],[]]
        
        return self.state1, self.state2, self.T
        
    def setRecord(self, kind):
        if kind == 1:
            self.enormalBikes = (self.enormalBikes * self.nt + self.normalBikes * (self.T - self.nt)) / self.T
            self.normalBikes, self.nt = self.normalBikes - 1, self.T
            self.eBP = (self.eBP * self.bpt + self.BP * (self.T - self.bpt)) / self.T
            self.BP, self.bpt = self.BP + 1, self.T
            self.hn[0].append(self.T)
            self.hn[1].append(self.enormalBikes/self.M)
        elif kind == 2:
            if self.RC == 0: 
                self.eidleRate = (self.eidleRate * self.it + (self.T - self.it)) / self.T
                self.hi[0].append(self.T)
                self.hi[1].append(self.eidleRate)
                self.it = self.T
            self.eBP = (self.eBP * self.bpt + self.BP * (self.T - self.bpt)) / self.T
            self.BP, self.bpt = self.BP - min(self.B_, self.state1[-3]), self.T
            self.eRC = (self.eRC * self.rct + self.RC * (self.T - self.rct)) / self.T
            self.RC, self.rct = self.RC + min(self.B_, self.state1[-3]), self.T
        elif kind == 3:
            self.eRC = (self.eRC * self.rct + self.RC * (self.T - self.rct)) / self.T
            self.RC, self.rct = self.RC - 1, self.T
            self.eDP = (self.eDP * self.dpt + self.DP * (self.T - self.dpt)) / self.T
            self.DP, self.dpt = self.DP + 1, self.T
            if self.RC == 0: 
                self.eidleRate = (self.eidleRate * self.it) / self.T
                self.hi[0].append(self.T)
                self.hi[1].append(self.eidleRate)
                self.it = self.T
        elif kind == 4: 
            self.enormalBikes = (self.enormalBikes * self.nt + self.normalBikes * (self.T - self.nt)) / self.T
            self.normalBikes, self.nt = self.normalBikes + min(self.D_, self.state1[-1]), self.T
            self.eDP = (self.eDP * self.dpt + self.DP * (self.T - self.dpt)) / self.T
            self.DP, self.dpt = self.DP - min(self.D_, self.state1[-1]), self.T
            self.hn[0].append(self.T)
            self.hn[1].append(self.enormalBikes/self.M)
        elif kind == -10:
            self.arrivals += 1
            self.hl[0].append(self.T)
            self.hl[1].append(self.lostCustomers/self.arrivals)
        elif kind == -11:
            self.lostCustomers += 1
            self.hl[0].append(self.T)
            self.hl[1].append(self.lostCustomers/self.arrivals)
        else: print('fucking wrong')
    
    def simulate(self):
        if WriteFile:
            with open(FileAdd, 'w') as fout:
                writer = csv.writer(fout)
                for i in range(EPISODES):
                    self.reset()
                    self.epi = i
                    while self.T <= self.timeLimit:
                        # print(self.T)
                        self.stepForward()
                writer.writerow([self.normalBikes, self.brokenBikes, self.lostCustomers/self.arrivals, self.idleRate, self.BP, self.RC, self.DP])  
        else:
            EPISODES = 1
            s = time.time()
            for i in range(EPISODES):
                self.epi = i
                self.reset()
                #plt.ion()
                while self.T <= self.timeLimit:
                    while self.T <= self.timeLimit:
                        self.stepForward()
                    #plt.clf()
                    plt.figure()
                    plt.plot(self.hn[0], self.hn[1], )
                    plt.plot(self.hi[0], self.hi[1], )
                    plt.plot(self.hl[0], self.hl[1], )
                    plt.legend(['好车比例', '服务台空闲比例', '顾客损失比例'])
                    plt.title('A{}M{}系统性能参数-仿真时间图'.format(self.A, self.M))
                    plt.xlabel('仿真时间')
                    plt.ylabel('比例')
                    plt.pause(1)
                    self.timeLimit += int(input('more time:'))
                    #plt.pause(0.0001)
                    #self.setRecord()
                print([self.enormalBikes/self.M, self.lostCustomers/self.arrivals, self.eidleRate, self.eBP, self.eRC, self.eDP])
                print(np.mean(self.hn[1][-1000:]),np.mean(self.hl[1][-1000:]),np.mean(self.hi[1][-1000:]))
            e = time.time()
            print('-'*30, '\n', e-s)


    def addEvent(self, kind):
        if kind == -1:
            next_time = random.expovariate(self.ArrLst[self.start]) + self.T
            start, end = self.start, self.start
        elif kind == 1:
            next_time = random.expovariate(self.RhoMtx[self.start][self.terminal]) + self.T
            start, end = self.start, self.terminal
        elif kind == 2: 
            next_time = random.expovariate(self.Gamma)
            next_time += self.T
            if self.state1[-3] >= self.B_: B = self.B_
            else: B = self.state1[-3]
            #B = self.state1[-3]
            start, end = B, 'f'
            #print('add event 2')
        elif kind == 3:
            next_time = random.expovariate(self.Mu) 
            if self.state1[-2] < self.N:
                next_time += self.T 
                heapq.heappush(self.F, next_time)
            else: 
                next_time += heapq.heappop(self.F)
                heapq.heappush(self.F, next_time)
            start, end = 'f', 'd'
        else: 
            next_time = random.expovariate(self.Delta)
            next_time += self.T
            if self.state1[-1] >= self.D_: D = self.D_
            else: D = self.state1[-1]
            #D = self.state1[-1]
            start, end = D, 'ni'
        heapq.heappush(self.scheduler, [next_time, kind, start, end])
    
    def assertion(self):
        su = 0
        S = list(self.state1)
        S.extend(list(np.array(self.state2).reshape(-1)))
        for s in S:
            assert(s>=0 and s<=self.M)
            su += s
        assert su == self.M
    
    def bikeArr(self):
        self.state2[self.start][self.terminal] -= 1
        heapq.heappop(self.scheduler)
        if random.random()<self.Beta:
            self.state1[-3] += 1
            self.setRecord(1)
            if self.state1[-3] == self.B_ or self.normalBikes==0:
                # here above the conditions are important:
                # for the first: it's ==, so when there are more broken bikes
                # for the second: when the # of nb is 0
                # so there won't be more broken bikes
                self.addEvent(2)
        else:
            self.state1[self.terminal] += 1
        self.assertion()
    def BPover(self):
        heapq.heappop(self.scheduler)
        self.setRecord(2)
        for i in range(self.B_): 
            if self.state1[-3]>=1:
                self.state1[-3] -= 1
                self.addEvent(3) 
                self.state1[-2] += 1
        if self.state1[-3] >= self.B_: self.addEvent(2) # # of nb is 0 has been dealt before
        self.assertion()
    def repair(self):
        heapq.heappop(self.scheduler)
        if self.state1[-2] <= self.N: heapq.heappop(self.F)
        self.state1[-2] -= 1
        self.state1[-1] += 1
        self.setRecord(3)
        if self.state1[-1] == self.D_ or self.normalBikes==0:
            self.addEvent(4)
        self.assertion()
    def DPover(self):
        heapq.heappop(self.scheduler)
        self.setRecord(4)
        dbar = min(self.state1[-1], self.D_)
        for i in self.arr_rank: 
            if self.state1[i]>=self.num_dis[i]: continue
            else: 
                alloc_n = min(self.num_dis[i]-self.state1[i], dbar)
                self.state1[i] += alloc_n
                self.state1[-1] -= alloc_n
                dbar -= alloc_n
                if dbar == 0: break
        assert dbar == 0 
        if self.state1[-1] >= self.D_: self.addEvent(4)
        self.assertion()
    def cusArr(self):
        #print(self.state1, self.state2)
        #print('------------------------')
        self.setRecord(-10)
        if self.state1[self.start] == 0:  # 但没车
            heapq.heappop(self.scheduler)
            self.addEvent(-1)
            self.setRecord(-11)
        else:
            heapq.heappop(self.scheduler)
            self.addEvent(-1)
            # below use self.terminal to represent the target
            self.terminal = random.choices(self.areas, weights=Pij[self.start], k=1)[0]
            self.state1[self.start] -= 1
            self.state2[self.start][self.terminal] += 1 
            self.addEvent(1)
        self.assertion()

    def stepForward(self):
        event = self.scheduler[0]
        #print(event)
        self.T, self.kind, self.start, self.terminal = event[0], event[1], event[2], event[3]
        '''
        kind of events:
        -1: customer ride a bike away
         1: a bike arrives at any area
         2: BP greater than B_
         3: a bike is fixed
         4: DP greater than D_
        '''
        if self.kind == 1: 
            self.bikeArr() # 顾客骑行到达
        elif self.kind == 2:
            self.BPover() # 坏车达到阈值
        elif self.kind == 3:
            self.repair() # 修好一辆车
        elif self.kind == 4:
            self.DPover() # 再分配
        else:# 顾客到达
            self.cusArr() #顾客到达

        return self.state1, self.state2, self.T

In [5]:
#B_ = 5
env = Model(A, M, Pij, ArrLst, RhoMtx, Beta, B_, Gamma, Mu, N, Delta, D_)
%time env.simulate()

NameError: name 'B_' is not defined

In [27]:
A, M, D_ = 8,20,2
ArrLst = np.log1p(np.random.rand(A))
def get_target_number(A, M, D_, ArrLst):
    arr = np.array(ArrLst)/sum(ArrLst)
    num_dis = [int(M*x) for x in arr]
    left = M - sum(num_dis)
    for i in np.argsort(ArrLst)[:left]:
        num_dis[i] += 1
    return num_dis
num_dis = get_target_number(A, M, D_, ArrLst)
arr_rank = np.argsort(ArrLst)[::-1]
print(num_dis, '\n', ArrLst, '\n', arr_rank)

[3, 4, 2, 1, 3, 2, 3, 2] 
 [0.58014979 0.62395213 0.15970558 0.07227283 0.47046772 0.15526907
 0.55007336 0.34248545] 
 [1 0 6 4 7 2 5 3]


In [33]:
Instance = 4
np.random.seed(Instance)
random.seed(0)
A = 2
M = 8
def getPij(a):
    temp = np.log1p(np.random.rand(A, A))
    return (temp / sum(temp)).T
Pij = getPij(A)
ArrLst = np.log1p(np.random.rand(A))
Beta = 0.4
Gamma = 1
Mu = 1
N = 1
Delta = 1
B_, D_ = 2, 2
RhoMtx = [[1.0, 0.5],
          [0.5, 1.0]]
# RhoMtx = [[1/(abs(j-i)+1) for i in range(A)] for j in range(A)]

In [34]:
# test the simulation length needed

class Model():
    '''
    This is the central model
    '''
    # initiate the parameters in this function
    def __init__(self, A, M, Pij, ArrLst, RhoMtx, Beta, B_, Gamma, Mu, N, Delta, D_):     
        self.timeLimit = 100000
        self.areas = list(range(A))
        #self.epi = 0
        
        #self.Performance = [0] * EPISODES
        self.A, self.M, self.Pij, self.ArrLst, self.RhoMtx, self.Beta, self.B_, self.Gamma, self.Mu, self.N, self.Delta, self.D_ = \
            A, M, Pij, ArrLst, RhoMtx, Beta, B_, Gamma, Mu, N, Delta, D_
        
        self.color = ['r']
        
        def get_target_number():
            A, M, D_, ArrLst = self.A, self.M, self.D_, self.ArrLst
            arr = np.array(ArrLst)/sum(ArrLst)
            num_dis = [int(M*x) for x in arr]
            left = M - sum(num_dis)
            for i in np.argsort(ArrLst)[:left]:
                num_dis[i] += 1
            return num_dis
        self.num_dis = get_target_number()
        self.arr_rank = np.argsort(ArrLst)[::-1]
        #print(self.num_dis, self.ArrLst, self.arr_rank)
    def reset(self):
        self.T = 0 # time cursor
        self.formerT = 0
        
        # serve performance parameters
        self.normalBikes, self.brokenBikes = M, 0
        # maintainance performance parameters
        self.idleRate, self.BP, self.RC, self.DP = 0, 0, 0, 0
        self.arrivals, self.lostCustomers = 0, 0
        # indicators of time stamp
        [self.nt, self.bt, self.it, self.bpt, self.rct, self.dpt] = [0]*6
        self.enormalBikes, self.ebrokenBikes = 0, 0
        # maintainance performance parameters
        self.eidleRate, self.eBP, self.eRC, self.eDP = 0, 0, 0, 0
        self.Result = [0.0] * 7
        
        self.state1 = [int(M/A)]*A + [0]*3 # Nis, BP, FC, and DP
        self.state2 = [[0]*A for i in range(A)] # Rijs
        self.F = [] # time to be empty for fixing queue
        heapq.heapify(self.F)
        
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(A):
            heapq.heappush(self.scheduler, [random.expovariate(ArrLst[i]), -1, i, i])
        self.stateRecord = self.state1[:A] + self.state2[0] + self.state2[1] + self.state1[-3:]
        
        # save state for drawing
        self.hn, self.hl, self.hi = [], [], [0]
        
        return self.state1, self.state2, self.T
        
    def setRecord(self, kind):
        if kind == 1:
            self.enormalBikes = (self.enormalBikes * self.nt + self.normalBikes * (self.T - self.nt)) / self.T
            self.normalBikes, self.nt = self.normalBikes - 1, self.T
            self.eBP = (self.eBP * self.bpt + self.BP * (self.T - self.bpt)) / self.T
            self.BP, self.bpt = self.BP + 1, self.T
            if self.T > 20000:
                self.hn.append(self.enormalBikes/self.M)
        elif kind == 2:
            if self.RC == 0: 
                self.eidleRate = (self.eidleRate * self.it + (self.T - self.it)) / self.T
                if self.T > 20000:
                    self.hi.append(self.eidleRate)
                self.it = self.T
            self.eBP = (self.eBP * self.bpt + self.BP * (self.T - self.bpt)) / self.T
            self.BP, self.bpt = self.BP - min(self.B_, self.state1[-3]), self.T
            self.eRC = (self.eRC * self.rct + self.RC * (self.T - self.rct)) / self.T
            self.RC, self.rct = self.RC + min(self.B_, self.state1[-3]), self.T
        elif kind == 3:
            self.eRC = (self.eRC * self.rct + self.RC * (self.T - self.rct)) / self.T
            self.RC, self.rct = self.RC - 1, self.T
            self.eDP = (self.eDP * self.dpt + self.DP * (self.T - self.dpt)) / self.T
            self.DP, self.dpt = self.DP + 1, self.T
            if self.RC == 0: 
                self.eidleRate = (self.eidleRate * self.it) / self.T
                if self.T > 20000:
                    self.hi.append(self.eidleRate)
                self.it = self.T
        elif kind == 4: 
            self.enormalBikes = (self.enormalBikes * self.nt + self.normalBikes * (self.T - self.nt)) / self.T
            self.normalBikes, self.nt = self.normalBikes + min(self.D_, self.state1[-1]), self.T
            self.eDP = (self.eDP * self.dpt + self.DP * (self.T - self.dpt)) / self.T
            self.DP, self.dpt = self.DP - min(self.D_, self.state1[-1]), self.T
            if self.T > 20000:
                self.hn.append(self.enormalBikes/self.M)
        elif kind == -10:
            self.arrivals += 1
            if self.T > 20000:
                self.hl.append(self.lostCustomers/self.arrivals)
        elif kind == -11:
            self.lostCustomers += 1
            if self.T > 20000:
                self.hl.append(self.lostCustomers/self.arrivals)
        else: print('fucking wrong')
    
    def simulate(self):
        self.reset()
        #plt.ion()
        while self.T <= self.timeLimit:
            self.stepForward()
        re = [[np.mean(self.hn), np.std(self.hn)], 
               [np.mean(self.hl), np.std(self.hl)], 
                [np.mean(self.hi), np.std(self.hi)],
             self.eBP/self.M, self.eRC/self.M, self.eDP/self.M]
        return re

    def addEvent(self, kind):
        if kind == -1:
            next_time = random.expovariate(self.ArrLst[self.start]) + self.T
            start, end = self.start, self.start
        elif kind == 1:
            next_time = random.expovariate(self.RhoMtx[self.start][self.terminal]) + self.T
            start, end = self.start, self.terminal
        elif kind == 2: 
            next_time = random.expovariate(self.Gamma)
            next_time += self.T
            if self.state1[-3] >= self.B_: B = self.B_
            else: B = self.state1[-3]
            #B = self.state1[-3]
            start, end = B, 'f'
            #print('add event 2')
        elif kind == 3:
            next_time = random.expovariate(self.Mu) 
            if self.state1[-2] < self.N:
                next_time += self.T 
                heapq.heappush(self.F, next_time)
            else: 
                next_time += heapq.heappop(self.F)
                heapq.heappush(self.F, next_time)
            start, end = 'f', 'd'
        else: 
            next_time = random.expovariate(self.Delta)
            next_time += self.T
            if self.state1[-1] >= self.D_: D = self.D_
            else: D = self.state1[-1]
            #D = self.state1[-1]
            start, end = D, 'ni'
        heapq.heappush(self.scheduler, [next_time, kind, start, end])
    
    def assertion(self):
        su = 0
        S = list(self.state1)
        S.extend(list(np.array(self.state2).reshape(-1)))
        for s in S:
            assert(s>=0 and s<=self.M)
            su += s
        assert su == self.M
    
    def bikeArr(self):
        self.state2[self.start][self.terminal] -= 1
        heapq.heappop(self.scheduler)
        if random.random()<self.Beta:
            self.state1[-3] += 1
            self.setRecord(1)
            if self.normalBikes==0:
                self.addEvent(2)
                self.addEvent(4)
            elif self.state1[-3] == self.B_:
                # here above the conditions are important:
                # for the first: it's ==, so when there are more broken bikes
                # for the second: when the # of nb is 0
                # so there won't be more broken bikes
                self.addEvent(2)
        else:
            self.state1[self.terminal] += 1
    def BPover(self):
        heapq.heappop(self.scheduler)
        self.setRecord(2)
        for i in range(self.B_): 
            if self.state1[-3]>=1:
                self.state1[-3] -= 1
                self.addEvent(3) 
                self.state1[-2] += 1
        if self.state1[-3] >= self.B_: self.addEvent(2) # # of nb is 0 has been dealt before
    def repair(self):
        heapq.heappop(self.scheduler)
        if self.state1[-2] <= self.N: heapq.heappop(self.F)
        self.state1[-2] -= 1
        self.state1[-1] += 1
        self.setRecord(3)
        if self.state1[-1] == self.D_:
            self.addEvent(4)
    def DPover(self):
        heapq.heappop(self.scheduler)
        self.setRecord(4)
        dbar = min(self.state1[-1], self.D_)
        for i in self.arr_rank: 
            if self.state1[i]>=self.num_dis[i]: continue
            else: 
                alloc_n = min(self.num_dis[i]-self.state1[i], dbar)
                self.state1[i] += alloc_n
                self.state1[-1] -= alloc_n
                dbar -= alloc_n
                if dbar == 0: break
        if self.state1[-1] >= self.D_: self.addEvent(4)
    def cusArr(self):
        self.setRecord(-10)
        if self.state1[self.start] == 0:  # 但没车
            heapq.heappop(self.scheduler)
            self.addEvent(-1)
            self.setRecord(-11)
        else:
            heapq.heappop(self.scheduler)
            self.addEvent(-1)
            # below use self.terminal to represent the target
            self.terminal = random.choices(self.areas, weights=Pij[self.start], k=1)[0]
            self.state1[self.start] -= 1
            self.state2[self.start][self.terminal] += 1 
            self.addEvent(1)

    def stepForward(self):
        event = self.scheduler[0]
        self.T, self.kind, self.start, self.terminal = event[0], event[1], event[2], event[3]
        '''
        kind of events:
        -1: customer ride a bike away
         1: a bike arrives at any area
         2: BP greater than B_
         3: a bike is fixed
         4: DP greater than D_
        '''
        if self.kind == 1: 
            self.bikeArr() # 顾客骑行到达
        elif self.kind == 2:
            self.BPover() # 坏车达到阈值
        elif self.kind == 3:
            self.repair() # 修好一辆车
        elif self.kind == 4:
            self.DPover() # 再分配
        else:# 顾客到达
            self.cusArr() #顾客到达

        return self.state1, self.state2, self.T

In [35]:
#B_ = 5
env = Model(A, M, Pij, ArrLst, RhoMtx, Beta, B_, Gamma, Mu, N, Delta, D_)
%time env.simulate()

CPU times: user 1.59 s, sys: 11 ms, total: 1.6 s
Wall time: 1.65 s


[[0.7883029517496433, 0.01715534688544253],
 [0.21705420172116943, 0.00488605521324889],
 [0.7714661790094177, 0.006146941618416982],
 0.09372604790301971,
 0.046941204128906316,
 0.08452850113978141]

In [38]:
np.random.seed(0)
def getPij(a):
    temp = np.log1p(np.random.rand(A,A))
    return (temp/sum(temp)).T
Pij = getPij(A)
ArrLst = np.random.rand(A)
B_ = np.random.choice(range(1,7))
Gamma = np.random.rand()
D_ = np.random.choice(range(1,7))
Delta = np.random.rand()
N = np.random.choice(range(1,7))
Mu = np.random.rand()
instance = Model(A, M, Pij, ArrLst, RhoMtx, Beta, B_, Gamma, Mu, N, Delta, D_)
print(instance.simulate())

[[0.49229901635115425, 0.0004226525862779459], [0.32397006791282235, 0.0016571677934646934], [0.7029212475766228, 0.007625594011884724], 0.34711904861480236, 0.09140435854709025, 0.06893821576188304]
